In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import time

In [2]:
from ema_workbench import (Model, RealParameter, ScalarOutcome, MultiprocessingEvaluator, 
                           ema_logging, Constant, Scenario, Policy, perform_experiments)
from ema_workbench import load_results
from ema_workbench.analysis import prim, dimensional_stacking, cart
from ema_workbench.util import ema_logging, utilities

from SALib.analyze import sobol
from ema_workbench.em_framework.evaluators import Samplers # 
from ema_workbench.em_framework.salib_samplers import get_SALib_problem

ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

In [3]:
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation

dike_model,  planning_steps = get_model_for_problem_formulation(5) 

In [4]:
policies = [Policy("base case without policy", **{k.name: 0 for k in dike_model.levers})]
problem = get_SALib_problem(dike_model.uncertainties)
print(problem)

{'num_vars': 19, 'names': ['A.0_ID flood wave shape', 'A.1_Bmax', 'A.1_Brate', 'A.1_pfail', 'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 'A.3_Bmax', 'A.3_Brate', 'A.3_pfail', 'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 'A.5_Bmax', 'A.5_Brate', 'A.5_pfail', 'discount rate 0', 'discount rate 1', 'discount rate 2'], 'bounds': [(0, 133), (30, 350), (0, 3), (0, 1), (30, 350), (0, 3), (0, 1), (30, 350), (0, 3), (0, 1), (30, 350), (0, 3), (0, 1), (30, 350), (0, 3), (0, 1), (0, 4), (0, 4), (0, 4)]}


In [5]:
with MultiprocessingEvaluator(dike_model) as evaluator:
    sobol_results = evaluator.perform_experiments(scenarios=1000, policies = policies,
                                               uncertainty_sampling=Samplers.SOBOL)
    
experiments, outcomes = sobol_results


[MainProcess/INFO] pool started with 8 workers
/opt/anaconda3/lib/python3.9/site-packages/SALib/sample/saltelli.py:94: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (1000) is equal to `2^n`.
        
  warnings.warn(msg)
[MainProcess/INFO] performing 40000 scenarios * 1 policies * 1 model(s) = 40000 experiments
  0%|                                     | 1/40000 [00:03<37:25:12,  3.37s/it]Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/ema_workbench/em_framework/callbacks.py", line 246, in _store_outcomes
    self.results[outcome][case_id,] = outcome_res
KeyError: 'A.1_Dike Investment Costs 0'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/ema_workbench/em_framework/ema_multiprocessing.py", line 253, in run
    self.callback(*result.get())
  File "/opt/anaconda3/lib/python3.9/site-packages/e

KeyboardInterrupt: 

In [ ]:
# function to aggregate over time and locations
def aggregate_df(df):
    df_aggregate_time = pd.DataFrame()
    df_aggregate_time_location = pd.DataFrame()
    locations = ['A.1', 'A.2', 'A.3', 'A.4', 'A.5']
    step = ['0', '1', '2']
    metrics = ['Expected Annual Damage', 'Dike Investment Costs', 'Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs']
    
    for metric in metrics:
        if metric == "RfR Total Costs" or metric == "Expected Evacuation Costs":

            columns = [metric + ' ' + time for time in step]

            df_aggregate_time_location[metric + ' time aggregate'] = df[columns].sum(axis=1)
        else:
            for location in locations:
                columns = [location + '_' + metric + ' ' + time for time in step]

                df_aggregate_time[location + '_' + metric + ' time aggregate'] = df[columns].sum(axis=1)
            
            columns_locations = [location + '_' + metric + ' time aggregate' for location in locations]
            df_aggregate_time_location[metric + ' time location aggregate'] = df_aggregate_time[columns_locations].sum(axis=1)
                    

    return df_aggregate_time, df_aggregate_time_location

In [ ]:
df_t, df_tl = aggregate_df(outcomes_df)
df_tl

In [ ]:
# outcomes_df = pd.DataFrame(outcomes)
# experiments_df = pd.DataFrame(experiments)

In [ ]:
x1 = df_tl['Expected Annual Damage time location aggregate'].to_numpy()
x2 = df_tl['Expected Number of Deaths time location aggregate'].to_numpy()

In [ ]:
Si_Damage = sobol.analyze(problem, x1, 
                          calc_second_order=True, print_to_console=False)

In [ ]:
scores_filtered = {k: Si_Damage[k] for k in ["ST", "ST_conf", "S1", "S1_conf"]}
Si_Damage_df = pd.DataFrame(scores_filtered, index=problem["names"])

sns.set_style("white")
fig, ax = plt.subplots(1)


indices = Si_Damage_df[["S1", "ST"]]
err = Si_Damage_df[["S1_conf", "ST_conf"]]

indices.plot.bar(yerr=err.values.T, ax=ax)
fig.set_size_inches(12, 10)
fig.subplots_adjust(bottom=0.3)
plt.show()

In [ ]:
Si_Deaths = sobol.analyze(problem, x2,
                  calc_second_order=True, print_to_console=False)

In [ ]:
scores_filtered = {k: Si_Deaths[k] for k in ["ST", "ST_conf", "S1", "S1_conf"]}
Si_Deaths_df = pd.DataFrame(scores_filtered, index=problem["names"])

sns.set_style("white")
fig, ax = plt.subplots(1)

indices = Si_Deaths_df[["S1", "ST"]]
err = Si_Deaths_df[["S1_conf", "ST_conf"]]

indices.plot.bar(yerr=err.values.T, ax=ax)
fig.set_size_inches(12, 10)
fig.subplots_adjust(bottom=0.3)
plt.show()

In [ ]:
df_t

In [ ]:
x3 = df_t['A.1_Expected Annual Damage time aggregate'].to_numpy()
x4 = df_t['A.1_Expected Number of Deaths time aggregate'].to_numpy()

In [ ]:
Si_Damage_l = sobol.analyze(problem, x3, 
                          calc_second_order=True, print_to_console=False)

In [ ]:
scores_filtered = {k: Si_Damage_l[k] for k in ["ST", "ST_conf", "S1", "S1_conf"]}
Si_Damage_df = pd.DataFrame(scores_filtered, index=problem["names"])

sns.set_style("white")
fig, ax = plt.subplots(1)


indices = Si_Damage_df[["S1", "ST"]]
err = Si_Damage_df[["S1_conf", "ST_conf"]]

indices.plot.bar(yerr=err.values.T, ax=ax)
fig.set_size_inches(12, 10)
fig.subplots_adjust(bottom=0.3)
plt.show()

In [ ]:
Si_Deaths_l = sobol.analyze(problem, x4,
                  calc_second_order=True, print_to_console=False)

In [ ]:
scores_filtered = {k: Si_Deaths_l[k] for k in ["ST", "ST_conf", "S1", "S1_conf"]}
Si_Deaths_df = pd.DataFrame(scores_filtered, index=problem["names"])

sns.set_style("white")
fig, ax = plt.subplots(1)

indices = Si_Deaths_df[["S1", "ST"]]
err = Si_Deaths_df[["S1_conf", "ST_conf"]]

indices.plot.bar(yerr=err.values.T, ax=ax)
fig.set_size_inches(12, 10)
fig.subplots_adjust(bottom=0.3)
plt.show()

In [ ]:
utilities.save_results(sobol_results, 'results/base_case_sobol_lever.csv')

In [ ]:
# sobol_results=utilities.load_results('results/base_case_sobol_lever.csv')
# experiments, outcomes = sobol_results
# outcomes = pd.DataFrame(outcomes)
# experiments = pd.DataFrame(experiments)

In [ ]:
outcomes


In [ ]:
experiments

In [ ]:
from SALib.analyze.sobol import analyze

In [ ]:
sobol_results = {}
for policy in experiments.policy.unique():
    logical = experiments.policy == policy
    y = outcomes['Expected Number of Deaths'][logical]
    indices = analyze(problem, y)
    sobol_results[policy] = indices

In [ ]:
indices

In [ ]:
plt.hist(y)
plt.show()

In [ ]:
from ema_workbench.analysis import feature_scoring

In [ ]:
cleaned_experiments = experiments.drop(columns=[l.name for l in dike_model.levers])

scores = feature_scoring.get_feature_scores_all(cleaned_experiments,sobol_results)